In [38]:
from random import randint
from math import gcd

def randuntil(lb, ub, until):
    while True:
        x = randint(lb, ub)
        if until(x): return x
        
def hexdump(xs):
    return ' '.join(f'{x:08X}' for x in xs)

def is_prime(n):
  if n == 2 or n == 3: return True
  if n < 2 or n%2 == 0: return False
  if n < 9: return True
  if n%3 == 0: return False
  r = int(n**0.5)
  # since all primes > 3 are of the form 6n ± 1
  # start with f=5 (which is prime)
  # and test f, f+2 for being prime
  # then loop by 6. 
  f = 5
  while f <= r:
    if n % f == 0: return False
    if n % (f+2) == 0: return False
    f += 6
  return True    

In [39]:
def generate_keys(p=None, q=None):
    # Wybierz losowo dwie duże liczby pierwsze
    if p is None: p = randuntil(2, 1e4-1, is_prime);                   print('p =', p)
    if q is None: q = randuntil(2, 1e4-1, is_prime);                   print('q =', q)
    assert p != q
    
    # Oblicz n
    n = p*q                                                         
    print('n =', n)
    
    # Oblicz wartość funkcji Eulera dla n
    ϕ = (p - 1)*(q - 1)
    print('ϕ =', ϕ)
    
    # Wybierz liczbę względnie pierwszą (najwiekszym wspólnym dzielnikiem jest jeden) z ϕ(n) taką że 1 < e < ϕ(n) 
    e = randuntil(3, 2**32, lambda x: gcd(x, ϕ) == 1)
    print('e =', e)
    
    # Znajdujemy liczbę d, gdzie jej różnica z odwrotnością modularną liczby e jest podzielna przez φ(n)
    d = pow(e, -1, ϕ)
    print('d =', d)

    # Zwróć klucz publiczny i klucz prywatny
    return (e, n), (d, n)

In [40]:
def cipher(key, msg):
    p, n = key
    # Szyfruj wiadomość c = x^p mod n
    return [pow(x, p, n) for x in msg]


encrypt = lambda k, m: cipher(k, bytes(m, 'utf-8'))
decrypt = lambda k, m: bytes(cipher(k, m)).decode('utf-8')

In [41]:
pub, prv = generate_keys()

p = 1693
q = 1217
n = 2060381
ϕ = 2057472
e = 263444513
d = 1743329


In [42]:
original = 'Tajny tekst do zaszyfrowania - oryginalna wersja 1'
assert len(bytes(original, 'utf-8')) == 50
print(original)

Tajny tekst do zaszyfrowania - oryginalna wersja 1


In [43]:
secret = encrypt(pub, original)
print('length =', len(secret), '\n'+'-'*11)
print(hexdump(secret))

length = 50 
-----------
0003B758 0004628D 0015F891 00173F45 0002DFA0 001F1D9D 0017B984 00178269 00163D1C 0000F004 0017B984 001F1D9D 001250AB 00194F24 001F1D9D 0016BDEF 0004628D 0000F004 0016BDEF 0002DFA0 000BAD00 0005C5E6 00194F24 000554F7 0004628D 00173F45 001865C4 0004628D 001F1D9D 00024020 001F1D9D 00194F24 0005C5E6 0002DFA0 001D8B10 001865C4 00173F45 0004628D 0000DDF3 00173F45 0004628D 001F1D9D 000554F7 00178269 0005C5E6 0000F004 0015F891 0004628D 001F1D9D 001C9197


In [44]:
recevied = decrypt(prv, secret)
assert recevied == original
print(recevied)

Tajny tekst do zaszyfrowania - oryginalna wersja 1
